In [1]:
import os
import sys
import json
import time
import datetime
from importlib import reload

import requests

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

import external as ext

In [2]:
desert_main = pd.read_csv('miramar.csv')
baltic_main = pd.read_csv('erangel.csv')
savage_main = pd.read_csv('sanhok.csv')
summerland_main = pd.read_csv('karakin.csv')

In [3]:
desert_main = desert_main.iloc[:,1:]
baltic_main = baltic_main.iloc[:,1:]
savage_main = savage_main.iloc[:,1:]
summerland_main = summerland_main.iloc[:,1:]

In [30]:
def risklevel(x,y,mapname):
    if mapname == 'Desert_Main':
        try:
            risk = desert_main.iloc[y,x]
        except:
            risk = 0
    elif mapname == 'Baltic_Main':
        try:
            risk = baltic_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Savage_Main':
        try:
            risk = savage_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Summerland_Main':
        try:
            risk = summerland_main.iloc[y,x]
        except:
            risk = 0
        # correction due to lower max number of players
        risk = risk * 1,5625
    lst.append(risk)

# <font color=red> Preparing DataFrame </font>

In [5]:
# RUN THIS CELL TO WORK WITH THE FILE UPLOADED IN THE REPOSITORY
# read files extracted from this as DataFrame
with open('telemetry_data/2020-04-09/pc/4c76d82b-90f7-4fd1-9ceb-97b99347e94e.json', 'r') as file:
    json_data = file.read()
    file.close()
data = []
head = []
for i in json.loads(json_data):
    if i["_T"] in ("LogPlayerPosition", "LogParachuteLanding"):
        data.append(i)
    if i["_T"] in ("LogMatchStart"):
        head.append(i)

In [6]:
# get events data into pandas dataframe
data = pd.read_json(json.dumps(data))
head = pd.read_json(json.dumps(head))
data.head()

,_D,_T,character,common,distance,elapsedTime,numAlivePlayers,vehicle
0,2020-04-08T22:33:24.153Z,LogPlayerPosition,"{'name': 'Wenurous', 'teamId': 4, 'health': 10...",{'isGame': 0},NaN,0.0,50.0,None
1,2020-04-08T22:33:24.220Z,LogPlayerPosition,"{'name': 'STSSEVEN', 'teamId': 11, 'health': 1...",{'isGame': 0},NaN,0.0,50.0,None
2,2020-04-08T22:33:24.220Z,LogPlayerPosition,"{'name': 'OMGNOWAYTHATSBS', 'teamId': 8, 'heal...",{'isGame': 0},NaN,0.0,50.0,None
3,2020-04-08T22:33:24.287Z,LogPlayerPosition,"{'name': 'LOCKLOAD007', 'teamId': 8, 'health':...",{'isGame': 0},NaN,0.0,50.0,None
4,2020-04-08T22:33:24.287Z,LogPlayerPosition,"{'name': 'Cadenn', 'teamId': 7, 'health': 100,...",{'isGame': 0},NaN,0.0,50.0,None


In [7]:
# get map name
mn = head.loc[0, 'mapName']

In [8]:
# copy match id to all rows and drop LogMatchDefinition event
# we will use this later to add use it for future merging tasks
matchid = json.loads(json_data)[0]["MatchId"]
data['MatchId'] = matchid

#reset indexes
data.reset_index(drop=True, inplace=True)
data.head()

,_D,_T,character,common,distance,elapsedTime,numAlivePlayers,vehicle,MatchId
0,2020-04-08T22:33:24.153Z,LogPlayerPosition,"{'name': 'Wenurous', 'teamId': 4, 'health': 10...",{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...
1,2020-04-08T22:33:24.220Z,LogPlayerPosition,"{'name': 'STSSEVEN', 'teamId': 11, 'health': 1...",{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...
2,2020-04-08T22:33:24.220Z,LogPlayerPosition,"{'name': 'OMGNOWAYTHATSBS', 'teamId': 8, 'heal...",{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...
3,2020-04-08T22:33:24.287Z,LogPlayerPosition,"{'name': 'LOCKLOAD007', 'teamId': 8, 'health':...",{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...
4,2020-04-08T22:33:24.287Z,LogPlayerPosition,"{'name': 'Cadenn', 'teamId': 7, 'health': 100,...",{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...


In [9]:
data.columns

Index(['_D', '_T', 'character', 'common', 'distance', 'elapsedTime',
       'numAlivePlayers', 'vehicle', 'MatchId'],
      dtype='object')

### Extracting Player information variables

In [10]:
# get player information
from pandas.io.json import json_normalize
# we need to expand the json documents on the character column
data = pd.concat([data, json_normalize(data['character'])], axis=1).drop(['character', 'zone'], axis=1)
data.head()

,_D,_T,common,distance,elapsedTime,numAlivePlayers,vehicle,MatchId,accountId,health,isInBlueZone,isInRedZone,location.x,location.y,location.z,name,ranking,teamId
0,2020-04-08T22:33:24.153Z,LogPlayerPosition,{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.534e426ebc5643d781264f68e02f8adb,100.0,False,False,84096.40625,730290.9375,5105.000000,Wenurous,0,4
1,2020-04-08T22:33:24.220Z,LogPlayerPosition,{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.c57f3b1dd9484b238b44a68f3321dfd2,100.0,False,False,718799.81250,546016.3125,-620.969971,STSSEVEN,0,11
2,2020-04-08T22:33:24.220Z,LogPlayerPosition,{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.54a138a899ea43919a51de65323d9287,100.0,False,False,491143.65625,634140.8750,12665.149414,OMGNOWAYTHATSBS,0,8
3,2020-04-08T22:33:24.287Z,LogPlayerPosition,{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.749c1f3bc22841ccb17fd9aa3e65bccf,100.0,False,False,490809.96875,634214.4375,12665.154297,LOCKLOAD007,0,8
4,2020-04-08T22:33:24.287Z,LogPlayerPosition,{'isGame': 0},NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.fa20a446eea64586b4033135908d921a,100.0,False,False,718356.50000,546602.7500,-701.961487,Cadenn,0,7


### Set isGame variable

In [11]:
def isGame(x):
    return x['isGame']

In [12]:
data['isGame'] = data['common'].apply(isGame)
data = data.drop('common', axis=1)
data.head()

,_D,_T,distance,elapsedTime,numAlivePlayers,vehicle,MatchId,accountId,health,isInBlueZone,isInRedZone,location.x,location.y,location.z,name,ranking,teamId,isGame
0,2020-04-08T22:33:24.153Z,LogPlayerPosition,NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.534e426ebc5643d781264f68e02f8adb,100.0,False,False,84096.40625,730290.9375,5105.000000,Wenurous,0,4,0.0
1,2020-04-08T22:33:24.220Z,LogPlayerPosition,NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.c57f3b1dd9484b238b44a68f3321dfd2,100.0,False,False,718799.81250,546016.3125,-620.969971,STSSEVEN,0,11,0.0
2,2020-04-08T22:33:24.220Z,LogPlayerPosition,NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.54a138a899ea43919a51de65323d9287,100.0,False,False,491143.65625,634140.8750,12665.149414,OMGNOWAYTHATSBS,0,8,0.0
3,2020-04-08T22:33:24.287Z,LogPlayerPosition,NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.749c1f3bc22841ccb17fd9aa3e65bccf,100.0,False,False,490809.96875,634214.4375,12665.154297,LOCKLOAD007,0,8,0.0
4,2020-04-08T22:33:24.287Z,LogPlayerPosition,NaN,0.0,50.0,None,match.bro.official.pc-2018-06.steam.squad.na.2...,account.fa20a446eea64586b4033135908d921a,100.0,False,False,718356.50000,546602.7500,-701.961487,Cadenn,0,7,0.0


In [13]:
# we will use the vehicle to check what players are still in the airplane
data.vehicle = data.vehicle.apply(lambda x: {} if pd.isna(x) else x)
data['vehicle'] = json_normalize(data.vehicle)['vehicleType']

### Interpolate players location for all elapsed times

In [14]:
# check how many players' locations we have at each point in time
# we want to make sure that after this process, all players have harmonized times and locations
for i in data.elapsedTime.unique():
    obs = len(data[data['elapsedTime'] == i])
    print(f'{i} has {obs} observations')

0.0 has 680 observations
1.0 has 21 observations
2.0 has 13 observations
3.0 has 5 observations
4.0 has 5 observations
5.0 has 7 observations
6.0 has 7 observations
7.0 has 1 observations
8.0 has 1 observations
9.0 has 6 observations
10.0 has 14 observations
11.0 has 22 observations
12.0 has 8 observations
13.0 has 5 observations
14.0 has 6 observations
15.0 has 6 observations
16.0 has 7 observations
17.0 has 1 observations
18.0 has 1 observations
19.0 has 6 observations
20.0 has 15 observations
21.0 has 22 observations
22.0 has 7 observations
23.0 has 6 observations
24.0 has 6 observations
25.0 has 5 observations
26.0 has 7 observations
27.0 has 1 observations
28.0 has 1 observations
29.0 has 7 observations
30.0 has 15 observations
31.0 has 21 observations
32.0 has 8 observations
33.0 has 5 observations
34.0 has 9 observations
35.0 has 4 observations
36.0 has 5 observations
37.0 has 1 observations
38.0 has 3 observations
39.0 has 6 observations
40.0 has 17 observations
41.0 has 19 obs

477.0 has 6 observations
478.0 has 4 observations
479.0 has 6 observations
480.0 has 5 observations
481.0 has 1 observations
482.0 has 1 observations
483.0 has 5 observations
484.0 has 10 observations
485.0 has 14 observations
486.0 has 6 observations
487.0 has 5 observations
488.0 has 6 observations
489.0 has 4 observations
490.0 has 5 observations
491.0 has 1 observations
492.0 has 1 observations
493.0 has 5 observations
494.0 has 15 observations
495.0 has 12 observations
496.0 has 4 observations
497.0 has 4 observations
498.0 has 6 observations
499.0 has 5 observations
500.0 has 3 observations
501.0 has 1 observations
502.0 has 3 observations
503.0 has 5 observations
504.0 has 15 observations
505.0 has 11 observations
506.0 has 3 observations
507.0 has 4 observations
508.0 has 7 observations
509.0 has 4 observations
510.0 has 3 observations
511.0 has 1 observations
512.0 has 3 observations
513.0 has 8 observations
514.0 has 15 observations
515.0 has 9 observations
516.0 has 3 observ

972.0 has 7 observations
973.0 has 1 observations
974.0 has 1 observations
976.0 has 3 observations
977.0 has 8 observations
978.0 has 13 observations
979.0 has 3 observations
980.0 has 2 observations
981.0 has 2 observations
982.0 has 7 observations
983.0 has 1 observations
984.0 has 1 observations
986.0 has 3 observations
987.0 has 8 observations
988.0 has 13 observations
989.0 has 3 observations
990.0 has 2 observations
991.0 has 3 observations
992.0 has 6 observations
993.0 has 1 observations
994.0 has 1 observations
996.0 has 3 observations
997.0 has 9 observations
998.0 has 11 observations
999.0 has 2 observations
1000.0 has 3 observations
1001.0 has 3 observations
1002.0 has 5 observations
1003.0 has 2 observations
1006.0 has 4 observations
1007.0 has 9 observations
1008.0 has 11 observations
1009.0 has 1 observations
1010.0 has 3 observations
1011.0 has 3 observations
1012.0 has 5 observations
1013.0 has 2 observations
1016.0 has 4 observations
1017.0 has 10 observations
1018.0

1599.0 has 6 observations
1600.0 has 1 observations
1601.0 has 1 observations
1602.0 has 1 observations
1603.0 has 3 observations
1608.0 has 4 observations
1609.0 has 5 observations
1610.0 has 1 observations
1611.0 has 1 observations
1612.0 has 1 observations
1613.0 has 3 observations
1618.0 has 4 observations
1619.0 has 4 observations
1620.0 has 1 observations
1621.0 has 1 observations
1622.0 has 1 observations
1623.0 has 3 observations
1628.0 has 2 observations
1629.0 has 4 observations
1630.0 has 1 observations
1631.0 has 1 observations
1632.0 has 1 observations
1633.0 has 3 observations
1638.0 has 3 observations
1639.0 has 3 observations
1640.0 has 1 observations
1641.0 has 1 observations
1642.0 has 2 observations
1643.0 has 2 observations
1648.0 has 3 observations
1649.0 has 4 observations
1651.0 has 1 observations
1652.0 has 3 observations
1653.0 has 1 observations
1658.0 has 4 observations
1659.0 has 1 observations
1660.0 has 1 observations
1661.0 has 1 observations
1662.0 has 2

In [15]:
# we create a new dataframe with the necessary information to interpolate locations
data_new_coords = data[['elapsedTime','name','location.x','location.y', 'vehicle']].sort_values(['name','elapsedTime']).drop_duplicates()
data_new_coords = data_new_coords.reset_index().drop('index', axis=1)
data_new_coords = data_new_coords.dropna(subset = ['elapsedTime'])
# we round coordinates for ease of calculations, and because a difference in location smaller than a cm is negligible
data_new_coords['location.x'] = data_new_coords['location.x'].apply(round)
data_new_coords['location.y'] = data_new_coords['location.y'].apply(round)
data_new_coords.head()

,elapsedTime,name,location.x,location.y,vehicle
0,0.0,10Baerini,719536,546556,NaN
1,1.0,10Baerini,254099,954480,TransportAircraft
2,11.0,10Baerini,258308,811172,TransportAircraft
3,21.0,10Baerini,262449,670174,TransportAircraft
4,30.0,10Baerini,266591,529161,TransportAircraft


In [16]:
data_new_coords.elapsedTime.unique().astype(int)

array([   0,    1,   11, ..., 1043, 1053, 1063])

In [17]:
%%time
# since different elapsed times don't show the location of all players, we want to interpolate the position of the players at each
# point in time. e.g. if player a is in position (x,y) at second 10.0 and in position (x+10,y+10) at second 20.0
# we can expect that this player will be at position (x+1,y+1) at second 11.0, and so on
def getNewCoord(t, n):
    aa = data_new_coords[(data_new_coords['elapsedTime'] < t) & (data_new_coords['name'] == n)] # get previous observations
    bb = data_new_coords[(data_new_coords['elapsedTime'] > t) & (data_new_coords['name'] == n)] # get following observations
    if (bool(aa['location.x'].any())) & (bool(bb['location.x'].any())): # if we have both previous and following observation
        a = aa.iloc[-1] # get last previous observation
        b = bb.iloc[0] # get first following observation
        time = b[0] - a[0] # calculate the time difference
        # get x and y coordinate by calculating movement speed in cm/s and get full coordinate
        new_coord_x = a[2] + ((b[2] - a[2]) / time)*(t-a[0])  
        new_coord_y = a[3] + ((b[3] - a[3]) / time)*(t-a[0])
        # we also get the vehicle to check if a player has jumped from the airplane or not
        in_aircraft = b['vehicle']
        return new_coord_x, new_coord_y, in_aircraft
    elif (not bool(aa['location.x'].any())) & (bool(bb['location.x'].any())):
        # if there is no previous observation, set first following location to be current location
        in_aircraft = b['vehicle']
        return b.iat[0,2], b.iat[0,3], in_aircraft

    # If a player doesn't have a future location, that indicates the player is dead
    elif (bool(aa['location.x'].any())) & (not bool(bb['location.x'].any())):
        # if there is no following observation, we assume the player has died (we hope that only in the match)
        return 'dead', 'dead', 'dead'

names=[]
times=[]
x_coords=[]
y_coords=[]
aircraft=[]
# we will get the location every 10 seconds from second 1 (e.g. 1.0, 11.0, 21.0), and at the last second
times_list = data_new_coords.elapsedTime.unique().astype(int)
# we get the times in intervals of 10s from second 1 to the last second of the game
unique_times_10s = np.arange(1, max(times_list), 10).tolist()
# we also append the last second, regardless of the 10s interval
unique_times_10s.append(max(times_list))
for t in unique_times_10s:
    for n in data_new_coords.name.unique():
        # for every player, we get their location at these times
        a = data_new_coords[(data_new_coords['elapsedTime'] == t) & (data_new_coords['name'] == n)]
        # if the player has a location, we take it
        if a['location.x'].any():
            names.append(n)
            times.append(t)
            x_coords.append(a.iat[0,2])
            y_coords.append(a.iat[0,3])
            aircraft.append(a['vehicle'])
        # if not, we interpolate it
        else:
            x_coord, y_coord, in_aircraft = getNewCoord(t,n)
            names.append(n)
            times.append(t)
            x_coords.append(x_coord)
            y_coords.append(y_coord)
            aircraft.append(in_aircraft)

CPU times: user 1min 36s, sys: 392 ms, total: 1min 37s
Wall time: 1min 37s


In [18]:
# we create the dataframe from these locations
players_location = pd.DataFrame(list(zip(times,names,x_coords, y_coords, aircraft)),
             columns=['time','name','location.x','location.y', 'inAircraft'])
# we also check if the players are or aren't in the aircraft with a binary variable
players_location['inAircraft'] = players_location['inAircraft'].apply(lambda x: 1 if str(x) == 'TransportAircraft' else 0)
players_location.head(10)

,time,name,location.x,location.y,inAircraft
0,1,10Baerini,254099,954480,0
1,1,2092,dead,dead,0
2,1,7Burial,626733,616448,1
3,1,7ooc_-_,254324,946841,1
4,1,8255,665941,572083,1
5,1,8344,254361,945559,1
6,1,91VinsmokeSanjii,dead,dead,0
7,1,AEPs4O4,254050,956144,0
8,1,AKA447,566028,662904,1
9,1,AeJiGooYo,dead,dead,0


In [19]:
# we make sure that harmonization of times and observations has been done
for i in players_location.time.unique():
    obs = len(players_location[players_location['time'] == i])
    print(f'{i} has {obs} observations')

1 has 100 observations
11 has 100 observations
21 has 100 observations
31 has 100 observations
41 has 100 observations
51 has 100 observations
61 has 100 observations
71 has 100 observations
81 has 100 observations
91 has 100 observations
101 has 100 observations
111 has 100 observations
121 has 100 observations
131 has 100 observations
141 has 100 observations
151 has 100 observations
161 has 100 observations
171 has 100 observations
181 has 100 observations
191 has 100 observations
201 has 100 observations
211 has 100 observations
221 has 100 observations
231 has 100 observations
241 has 100 observations
251 has 100 observations
261 has 100 observations
271 has 100 observations
281 has 100 observations
291 has 100 observations
301 has 100 observations
311 has 100 observations
321 has 100 observations
331 has 100 observations
341 has 100 observations
351 has 100 observations
361 has 100 observations
371 has 100 observations
381 has 100 observations
391 has 100 observations
401 has 100

### Get Collaboration information

In [20]:
# encode player names
# we do this to get the teamIds in the main dataframe
players = data[['name', 'accountId','teamId']].drop_duplicates()
players = players.sort_values('teamId').reset_index().drop(['index'], axis=1)
players

,name,accountId,teamId
0,dxnub,account.f41a6e2001664439b818784d15ffc124,1
1,Mac_1205,account.aae7ef8724284447953034a9ccbe89aa,1
2,spekture,account.6b46d60583254839bf6e8c3df814f115,1
3,Asin_Sira,account.154d4fd125a348e9a1f8df2daaabd9cb,1
4,HITECHPLUMBER69,account.08c1815b804c4b1494002f149c6d6601,2
5,Odmin,account.038f96e8563f4e03af57d48c684d7773,2
6,THE_IRONMAFIA,account.c7c9bf6dae124f5cb98cf581e0ce2c6d,2
7,Neal_No1,account.98a361b6ee494c9c8ae2c3f739415a84,2
8,RUCY06,account.3d2c03735ec84f6c9ecaa0554aafead3,3
9,WishLucy,account.ec1a36a3fd3c4847a2e7ea7e66696fde,3


In [21]:
# we merge these names to get accountId and teamId in the dataframe
players_location = pd.merge(players_location,
                            players,
                            left_on = 'name',
                            right_on = 'name',
                            how = 'left')
# moreover, since we are goint to later "asign" a number to the players (e.g. player1, player2), we want to make sure
# that the criteria is consistent for all teams. Therefore, we do it with the .sort_values method
# players on each team are ordered with this method inside the team
players_location = players_location[players_location['time'] >= 0].sort_values(['teamId','time', 'name']).reset_index().drop(['index'], axis=1)
players_location.head()

,time,name,location.x,location.y,inAircraft,accountId,teamId
0,1,Asin_Sira,254264,948862,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1
1,1,Mac_1205,254201,950985,0,account.aae7ef8724284447953034a9ccbe89aa,1
2,1,dxnub,443251,687020,1,account.f41a6e2001664439b818784d15ffc124,1
3,1,spekture,254157,952488,0,account.6b46d60583254839bf6e8c3df814f115,1
4,11,Asin_Sira,258465,805831,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1


In [22]:
players_location = players_location.replace('dead',0)

In [23]:
# convert to 10 meters from cms
players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x.astype(int))
players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x // 1000)
players_location.head()

,time,name,location.x,location.y,inAircraft,accountId,teamId
0,1,Asin_Sira,254,948,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1
1,1,Mac_1205,254,950,0,account.aae7ef8724284447953034a9ccbe89aa,1
2,1,dxnub,443,687,1,account.f41a6e2001664439b818784d15ffc124,1
3,1,spekture,254,952,0,account.6b46d60583254839bf6e8c3df814f115,1
4,11,Asin_Sira,258,805,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1


In [34]:
data = players_location[['location.x','location.y']]
data

,location.x,location.y
0,254,948
1,254,950
2,443,687
3,254,952
4,258,805
5,258,807
6,258,812
7,258,808
8,262,664
9,262,666


In [25]:
# convert df to list of tuples
records = data.to_records(index=False)
result = list(records)
result

[(254, 948),
 (254, 950),
 (443, 687),
 (254, 952),
 (258, 805),
 (258, 807),
 (258, 812),
 (258, 808),
 (262, 664),
 (262, 666),
 (262, 661),
 (262, 667),
 (266, 523),
 (266, 525),
 (266, 523),
 (266, 526),
 (282, 454),
 (283, 457),
 (293, 472),
 (268, 468),
 (327, 446),
 (331, 447),
 (333, 454),
 (272, 467),
 (358, 440),
 (352, 442),
 (353, 445),
 (296, 460),
 (366, 442),
 (363, 440),
 (362, 441),
 (288, 453),
 (366, 438),
 (370, 441),
 (365, 439),
 (288, 450),
 (366, 439),
 (369, 442),
 (366, 439),
 (296, 448),
 (366, 438),
 (370, 442),
 (365, 440),
 (296, 450),
 (365, 440),
 (369, 442),
 (365, 440),
 (298, 451),
 (366, 439),
 (369, 442),
 (365, 440),
 (296, 452),
 (366, 438),
 (367, 440),
 (365, 440),
 (294, 450),
 (366, 438),
 (365, 440),
 (365, 440),
 (296, 453),
 (366, 438),
 (365, 440),
 (365, 440),
 (305, 453),
 (366, 438),
 (365, 439),
 (365, 440),
 (327, 451),
 (366, 438),
 (365, 439),
 (365, 440),
 (343, 450),
 (366, 438),
 (365, 439),
 (0, 0),
 (358, 442),
 (366, 438),
 (3

In [35]:
cols = ['risk']
lst = []

for i in result:
    risklevel(*i, mapname=mn)

    
    
riskordered = pd.DataFrame(lst, columns=cols)
riskordered.head()

,risk
0,0
1,0
2,0
3,0
4,0


In [37]:
pd.concat([players_location, riskordered], axis=1, sort=False)

,time,name,location.x,location.y,inAircraft,accountId,teamId,risk
0,1,Asin_Sira,254,948,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
1,1,Mac_1205,254,950,0,account.aae7ef8724284447953034a9ccbe89aa,1,0
2,1,dxnub,443,687,1,account.f41a6e2001664439b818784d15ffc124,1,0
3,1,spekture,254,952,0,account.6b46d60583254839bf6e8c3df814f115,1,0
4,11,Asin_Sira,258,805,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
5,11,Mac_1205,258,807,0,account.aae7ef8724284447953034a9ccbe89aa,1,0
6,11,dxnub,258,812,1,account.f41a6e2001664439b818784d15ffc124,1,0
7,11,spekture,258,808,0,account.6b46d60583254839bf6e8c3df814f115,1,0
8,21,Asin_Sira,262,664,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
9,21,Mac_1205,262,666,0,account.aae7ef8724284447953034a9ccbe89aa,1,0


## Export to .csv

In [30]:
# export match telemetry data into .csv format
players_location.to_csv(path_or_buf=f'{matchid}.csv')